In [44]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bf

pd.options.mode.chained_assignment = None 
%matplotlib inline

In [7]:
urls = ['http://www.metacritic.com/critic/peter-travers?filter=movies&num_items=100&sort_options=metascore',
        'http://www.metacritic.com/critic/peter-travers?filter=movies&num_items=100&sort_options=metascore&page=1'
        'http://www.metacritic.com/critic/peter-travers?filter=movies&num_items=100&sort_options=metascore&page=4',
        'http://www.metacritic.com/critic/peter-travers?filter=movies&num_items=100&sort_options=metascore&page=6',
         'http://www.metacritic.com/critic/peter-travers?filter=movies&num_items=100&sort_options=metascore&page=11',
         'http://www.metacritic.com/critic/peter-travers?filter=movies&num_items=100&sort_options=metascore&page=13',
         'http://www.metacritic.com/critic/peter-travers?filter=movies&num_items=100&sort_options=metascore&page=17',
         'http://www.metacritic.com/critic/peter-travers?filter=movies&num_items=100&sort_options=metascore&page=16']

bf_list = []
for url in urls:
    request = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'})
    bf_list.append(bf(request.text, "lxml"))

In [8]:
reviews = []
movies = []
for bf_obj in bf_list:
    for i in range(len(bf_obj.find_all('li', class_="review_product_score brief_critscore"))):
        reviews.append(bf_obj.find_all('li', class_="review_product_score brief_critscore")[i].find('span').text)
        movies.append(bf_obj.find_all('div', class_="review_product")[i].find('a').text)

In [9]:
peter_travers_reviews = pd.DataFrame(dict(reviews = reviews, movies = movies))
len(peter_travers_reviews)

1700

In [14]:
peter_travers_reviews.to_pickle('peter_travers_movie_names_x_reviews.pkl')

NameError: name 'peter_travers_reviews' is not defined

In [3]:
from movies_module import imdb_clean 

In [ ]:
#Using Selenium to Scrape IMDB

chromedriver = "/Users/Fernanhid/Downloads/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver


driver = webdriver.Chrome(chromedriver)
driver.get("http://www.imdb.com/title/tt0499549/?ref_=nv_sr_1")
travis_bf_list = []

for title in peter_travers_reviews.movies:
    try:
        driver.find_element_by_id("navbar-query").clear()
        username_form = driver.find_element_by_id("navbar-query")
        username_form.send_keys(title)

        username_form.send_keys(Keys.RETURN)
        driver.implicitly_wait(20)
        #driver.find_elements_by_xpath("//*[@class='result_text']")[0].find_element_by_tag_name('a').click()
        driver.find_element_by_xpath('//*[@id="main"]/div/div[2]/table/tbody/tr[1]/td[2]/a').click()
        driver.implicitly_wait(10)


        #beautiful Soup Part

        page = requests.get(driver.current_url)
        page = bf(page.text)
        travis_bf_list.append(page)
    except:
        driver.find_element_by_id("navbar-query").clear()
        username_form = driver.find_element_by_id("navbar-query")
        username_form.send_keys('Avatar')
        username_form.send_keys(Keys.RETURN)


In [ ]:
#Having a Master Dictionary
travis_dict_movie = {}

#Iterating over the urls
for i, movie in enumerate(travis_df.movie_titles):
    try:
        #storing the content for each movie
        dict_content = {}
        #Iterating over all the content of the movie
        for div in  travis_bf_list[i].find('div', class_= 'article', id="titleDetails").find_all('div'):
            #Iterating Over Every part of the content
            try:
                #Preprosessing everyline
                process =  div.text.replace('\n', '').split(':')
                #Seperating the title and value
                key = process[0]
                content = process[1]
                #Adding the title and values in a dictionary
                dict_content[key] = content
            except:
                pass
    except:
        pass
    #Adding the content dictionary of one movie into the total dict
    travis_dict_movie[movie] = dict_content

## Features

In [ ]:
# Title

travis_movies_df['title'] = travis_movies_df.movie_titles.apply(lambda x: x.split('(')[0].strip())

In [ ]:
# Annual Gross

gross = []
for title in travis_movies_df.movie_titles:
    try:
        gross.append(imdb_clean.clean_cash(travis_dict_movie[title]['Gross']))
    except:
        gross.append('NaN')

travis_movies_df['gross'] = gross

In [12]:
# Release Date

release_date = []
for title in travis_movies_df.movie_titles:
    try:
        release_date.append(imdb_clean.clean_release(travis_dict_movie[title]['Release Date']))
    except:
        release_date.append('NaN')
        
travis_movies_df['release_date'] = release_date

In [ ]:
# Opening Weekend

opening_weekend = []
for title in travis_movies_df.movie_titles:
    try:
        opening_weekend.append(imdb_clean.clean_cash(travis_dict_movie[title]['Opening Weekend']))
    except:
        opening_weekend.append('NaN')
        
travis_movies_df['opening_weekend'] = opening_weekend

In [ ]:
# Runtime

run_time = []
for title in travis_movies_df.movie_titles:
    try:
        run_time.append(imdb_clean.clean_runtime(travis_dict_movie[title]['Runtime']))
    except:
        run_time.append('NaN')
        
travis_movies_df['run_time'] = run_time


In [ ]:
# Production Companies

prod_companies = []
for title in travis_movies_df.movie_titles:
    try:
        prod_companies.append(travis_dict_movie[title]['Production Co'])
    except:
        prod_companies.append('NaN')
        
travis_movies_df['production'] = prod_companies

In [ ]:
# Budget

budget = []
for title in travis_movies_df.movie_titles:
    try:
        budget.append(imdb_clean.clean_cash(travis_dict_movie[title]['Budget']))
    except:
        budget.append('NaN')
        
travis_movies_df['budget'] = budget

In [ ]:
# Country

country = []
for title in travis_movies_df.movie_titles:
    try:
        country.append(imdb_clean.clean_country(travis_dict_movie[title]['Country']))
    except:
        country.append('NaN')

travis_movies_df['country'] = country

travis_movies_df['usa_dummy'] = [1 if i == 'USA' else 0 for i in travis_movies_df.country]

In [ ]:
# Genre

genres = []
for title in travis_movies_df.movie_titles:
    try:
        genres.append(imdb_clean.clean_genre(travis_dict_movie_genre[title]['Genres']))
    except:
        genres.append('NaN')

# Clean Genres Cleans the name and         
travis_movies_df['main_genre'] = genres

In [ ]:
# Date
travis_movies_df['month_released'] = travis_movies_df.release_date.apply(lambda x: imdb_clean.get_month(x))
travis_movies_df['day_released'] = travis_movies_df.release_date.apply(lambda x: imdb_clean.get_day(x))
travis_movies_df['year_released'] = travis_movies_df.release_date.apply(lambda x: imdb_clean.get_year(x))

In [ ]:
travis_movies_df['reviews'] = peter_travers_reviews.reviews

In [22]:
travis_movies_df.head(3)

,movie_titles,gross,new_gross,release_date,new_opening_date,opening_weekend,new_opening_weekend,run_time,runtime_in_minutes,production,...,country,new_country,usa_dummy,title,genres,main_genre,month_released,day_released,year_released,reviews
0,Boyhood (2014) - IMDb,"$25,359,200 (USA)(27 Febr...",25359200,15 August 2014 (USA) See more »,2014-08-15 00:00:00,"$387,618 (USA) (11 ...",387618,165 min,165,"IFC Productions, Detour Filmproduction ...",...,USA,USA,1,Boyhood,Drama,Drama,August,Friday,2014,100
1,Pan's Labyrinth (2006) - IMDb,"$37,623,143 (USA)(8 June ...",37623143,19 January 2007 (USA) See more »,2007-01-19 00:00:00,"$501,691 (Mexico) (...",501691,118 min|112 min,118,"Estudios Picasso, Tequila Gang, ...",...,Spain|Mexico|USA,Spain,0,Pan's Labyrinth,Drama | Fantasy | War,Drama,January,Friday,2007,100
2,"4 Months, 3 Weeks and 2 Days (2007) - IMDb","$1,185,783 (USA)(2 May 2008)",1185783,14 September 2007 (Romania) See more »,2007-09-14 00:00:00,"$51,712 (USA) (25 J...",51712,113 min,113,"Mobra Films, Centrul National al Cinema...",...,Romania|Belgium,Romania,0,"4 Months, 3 Weeks and 2 Days",Drama,Drama,September,Friday,2007,100


### Making the Final DataFrame to be Analyzed

In [26]:
cleaner_df = travis_movies_df[(travis_movies_df.new_budget != 'NaN')&(travis_movies_df.new_opening_weekend != 'NaN')&(travis_movies_df.new_gross != 'NaN')&(travis_movies_df.reviews != 'no rating')&(travis_movies_df.runtime_in_minutes != 'NaN')&(travis_movies_df.new_opening_weekend != 'Nan')]
clean_sans_earnings_df = travis_movies_df[(travis_movies_df.new_budget != 'NaN')&(travis_movies_df.reviews != 'no rating')&(travis_movies_df.runtime_in_minutes != 'NaN')&(travis_movies_df.new_opening_weekend != 'Nan')]

In [28]:
cleaner_df.head(2)

,movie_titles,gross,new_gross,release_date,new_opening_date,opening_weekend,new_opening_weekend,run_time,runtime_in_minutes,production,...,country,new_country,usa_dummy,title,genres,main_genre,month_released,day_released,year_released,reviews
0,Boyhood (2014) - IMDb,"$25,359,200 (USA)(27 Febr...",25359200,15 August 2014 (USA) See more »,2014-08-15 00:00:00,"$387,618 (USA) (11 ...",387618,165 min,165,"IFC Productions, Detour Filmproduction ...",...,USA,USA,1,Boyhood,Drama,Drama,August,Friday,2014,100
3,12 Years a Slave (2013) - IMDb,"$56,667,870 (USA)(2 May 2...",56667870,8 November 2013 (USA) See more »,2013-11-08 00:00:00,"$923,715 (USA) (18 ...",923715,134 min,134,"Regency Enterprises, River Road Enterta...",...,USA|UK,USA,1,12 Years a Slave,Biography | Drama | History,Biography,November,Friday,2013,100


In [45]:
cleaner_df['reviews'] = cleaner_df['reviews'].apply(lambda x: int(x))
cleaner_df['runtime_in_minutes'] = cleaner_df['runtime_in_minutes'].apply(lambda x: int(x))
cleaner_df['new_budget'] = cleaner_df['new_budget'].apply(lambda x: int(x))
cleaner_df['new_gross'] = cleaner_df['new_gross'].apply(lambda x: int(x))
cleaner_df['new_opening_weekend'] = cleaner_df['new_opening_weekend'].apply(lambda x: int(x))

In [39]:
final_df = cleaner_df[['title',
                       'new_opening_date',
                         'reviews',
                       'month_released',
                       'new_opening_weekend',
                       'day_released',
                       'runtime_in_minutes',
                       'new_gross',
                       'new_budget', 
                       'usa_dummy',
                       'main_genre']][cleaner_df.title != 'The Dying Gaul']


In [40]:
final_df.head()

,title,new_opening_date,reviews,month_released,new_opening_weekend,day_released,runtime_in_minutes,new_gross,new_budget,usa_dummy,main_genre
0,Boyhood,2014-08-15 00:00:00,100,August,387618,Friday,165,25359200,4000000,1,Drama
3,12 Years a Slave,2013-11-08 00:00:00,100,November,923715,Friday,134,56667870,20000000,1,Biography
4,Ratatouille,2007-06-29 00:00:00,88,June,47027395,Friday,111,206435493,150000000,1,Animation
5,Gravity,2013-10-04 00:00:00,100,October,55785112,Friday,91,274084951,100000000,0,Adventure
6,Carol,2016-01-15 00:00:00,100,January,253510,Friday,118,253510,11800000,0,Drama


In [46]:
final_df.to_pickle('movies_travis_df_clean.pkl')